给定一个问题或者需求描述，生成多个方案，从中选择一个较为合适的。

In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.7, max_tokens=4096)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [3]:
template_rewriteq = '''你是个非常优秀的程序员，你的任务是根据用户的模糊描述，生成一个高质量的程序规格说明书。
1，这个说明书要非常详细，描述程序应该做什么，应该包含哪些功能和特征，并给出所有的细节；
2，列出程序的主要模块，可以使用算法名称及基本实现思路。

用户提供的描述: {raw_question}

请一步步思考，确保没有遗漏任何细节。这个说明书将被用以实现具体的程序。
生成的规格说明书：'''

prompt_rewriteq = PromptTemplate(input_variables=['raw_question'], template=template_rewriteq)
rewriteq_chain = LLMChain(llm=llm, prompt=prompt_rewriteq, output_key='question')

In [4]:
template_solution = '''你们是三个世界知名的软件架构师，你们的任务是根据程序规格说明书，提供初步的方案设计。
请你们每个人提供一个解决方案，并说明它解决了哪些问题，采用什么样的思路来解决，以及这个方案有什么样的优势。

程序说明书: {question}

请根据程序说明书，各自生成一个解决方案'''

prompt_solution = PromptTemplate(input_variables=['question'], template=template_solution)
solution_chain = LLMChain(llm=llm, prompt=prompt_solution, output_key='solution')

In [5]:
template_selection = '''你是一个著名的软件产品经理，请根据程序规格说明书从多个解决方案，选择一个最具创新性的解决方案。
该解决方案不仅能够通过代码实现出来，还要有一定的创新性，以便申请专利。请给出选择的最终方案，说明选择的理由，并重新描述这个解决方案的具体实现过程。
在描述解决方案时，需要包含所有的细节，以方便代码实现。

程序说明书: {question}
可选的解决方案: {solution}
请一步步思考，并给出最好的解决方案：'''

prompt_selection = PromptTemplate(input_variables=['question', 'solution'], template=template_selection)
selection_chain = LLMChain(llm=llm, prompt=prompt_selection, output_key='selection')

In [6]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[rewriteq_chain, solution_chain, selection_chain],
    input_variables=["raw_question"],
    # Here we return multiple variables
    output_variables=["question", "solution", "selection"],
    verbose=True)

In [7]:
# question = '''选择的创新潜力功能点是“租户转移”，它是指在租户管理员账号下，将当前租户转移到其他账号下进行管理。
# 目前市面上的流程挖掘和任务挖掘工具大多都没有这个功能，这个功能的创新点在于它能够帮助企业更好地管理租户，实现更加灵活的团队协作。
# 在新业务场景下或者选择新的算法，可以通过自动化转移流程、权限管理等方式来实现创新的解决方案。
# 同时，在租户转移过程中，系统需要对目标账号是否为当前租户的分析师角色进行校验，这也是一个创新点，可以提高系统的安全性和管理效率。
# '''

question = '''选择的创新潜力功能点是“租户转移”，它是指在租户管理员账号下，将当前租户转移到其他账号下进行管理。
目前市面上的流程挖掘和任务挖掘工具大多都没有这个功能，这个功能的创新点在于它能够帮助企业更好地管理租户，实现更加灵活的团队协作。
在新业务场景下或者选择新的算法，可以通过自动化转移流程、权限管理等方式来实现创新的解决方案。
同时，在租户转移过程中，系统需要对目标账号是否为当前租户的分析师角色进行校验，这也是一个创新点，可以提高系统的安全性和管理效率。
'''

result = overall_chain({'raw_question': question})



> Entering new SequentialChain chain...

> Finished chain.


In [8]:
print(result['question'])

一、程序功能和特征：

1.1 租户转移功能：租户管理员账号下，将当前租户转移到其他账号下进行管理。

1.2 自动化转移流程：实现租户转移的自动化流程，提高转移效率和管理效率。

1.3 权限管理：对租户转移的权限进行管理，确保安全性和管理效率。

1.4 目标账号分析师角色校验：在租户转移过程中，系统需要对目标账号是否为当前租户的分析师角色进行校验，提高系统的安全性和管理效率。

1.5 灵活的团队协作：租户转移功能能够帮助企业更好地管理租户，实现更加灵活的团队协作。

二、程序模块：

2.1 账号管理模块：管理租户管理员账号和其他账号的信息，包括用户名、密码、权限等。

2.2 租户管理模块：管理租户的信息，包括租户名称、租户管理员账号、租户成员账号等。

2.3 转移流程模块：实现租户转移的自动化流程，包括租户管理员账号的身份验证、目标账号的分析师角色校验、租户数据的备份和恢复等。

2.4 权限管理模块：对租户转移的权限进行管理，包括租户管理员账号的权限、目标账号的权限等。

2.5 安全性管理模块：对租户转移的安全性进行管理，包括身份验证、分析师角色校验、数据备份和恢复等。

2.6 团队协作模块：实现租户转移功能，帮助企业更好地管理租户，实现更加灵活的团队协作。


In [9]:
print(result['solution'])

架构师A的解决方案：

思路：我会采用微服务架构来解决此问题。将每个模块拆分成独立的服务，如账号管理服务、租户管理服务等。这些服务通过API进行通信，实现相互之间的协同工作。使用微服务架构的优势在于，每个服务都可以独立部署和维护，提高了系统的可靠性和可扩展性。

优势：

1. 高可靠性：由于每个服务都是独立的，一个服务的故障不会影响整个系统的运行。

2. 可扩展性：每个服务都可以根据需要进行扩展，避免了系统的单点故障问题。

3. 灵活性：微服务架构可以更好地支持团队协作，每个服务可以由不同的团队来负责，实现更加灵活的团队协作。

架构师B的解决方案：

思路：我会采用领域驱动设计来解决此问题。将每个模块拆分成领域对象，如账号对象、租户对象等。这些对象通过领域事件和领域服务进行协作，实现租户转移功能。使用领域驱动设计的优势在于，将业务逻辑和技术实现分离，更加关注业务问题而不是技术问题。

优势：

1. 高内聚性：每个领域对象都是具有高内聚性的，避免了模块之间的耦合性问题。

2. 可维护性：领域驱动设计将业务逻辑和技术实现分离，使得系统更加易于维护和扩展。

3. 高效性：领域驱动设计可以将系统的复杂度降到最低，并提高系统的效率。

架构师C的解决方案：

思路：我会采用事件驱动架构来解决此问题。将每个模块拆分成独立的组件，如账号管理组件、租户管理组件等。这些组件通过事件进行通信，实现相互之间的协同工作。使用事件驱动架构的优势在于，将模块之间的耦合度降到最低，提高了系统的可靠性和可扩展性。

优势：

1. 解耦性：事件驱动架构可以将模块之间的耦合度降到最低，避免了模块之间的相互依赖问题。

2. 可扩展性：事件驱动架构可以根据需要进行组件的扩展，避免了系统的单点故障问题。

3. 灵活性：事件驱动架构可以更好地支持团队协作，每个组件可以由不同的团队来负责，实现更加灵活的团队协作。


In [10]:
print(result['selection'])

经过综合考虑三个架构师的解决方案，我认为架构师B的领域驱动设计解决方案最具创新性，并且可以申请专利。

选择架构师B的解决方案的理由如下：

1. 高内聚性：领域驱动设计将每个模块拆分成领域对象，每个对象都具有高内聚性，避免了模块之间的耦合性问题。

2. 可维护性：领域驱动设计将业务逻辑和技术实现分离，使得系统更加易于维护和扩展。

3. 高效性：领域驱动设计可以将系统的复杂度降到最低，并提高系统的效率。

4. 创新性：领域驱动设计是一种全新的软件架构设计方式，可以申请专利。

重新描述架构师B的解决方案实现过程如下：

1. 定义领域对象：首先需要定义账号、租户等领域对象，并确定它们之间的关系。

2. 编写领域服务：编写实现租户转移功能的领域服务，包括身份验证、分析师角色校验、数据备份和恢复等。

3. 定义领域事件：定义租户转移时需要触发的领域事件，如“租户转移前备份数据”、“租户转移成功”等。

4. 通过领域事件实现协作：通过领域事件，各个领域对象之间进行协作，实现租户转移功能。

5. 安全性管理：在租户转移过程中，需要进行身份验证、分析师角色校验、数据备份和恢复等安全性管理措施。

6. 团队协作：领域驱动设计可以更好地支持团队协作，每个领域对象可以由不同的团队来负责，实现更加灵活的团队协作。

总之，架构师B的领域驱动设计解决方案可以更好地解决程序规格说明书中的问题，具有高内聚性、可维护性、高效性和创新性，可以申请专利。


In [11]:
with open('patents/specification-test4.txt', 'w') as f:
    f.write('question: ' + result['raw_question'])    
    f.write('\n\nsolution: ' + result['solution'])
    f.write('\n\nselection: ' + result['selection'])